# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


## Calculate utility score of training set where `action` = 1 for all `resp` > 0
[training set](https://www.kaggle.com/c/jane-street-market-prediction/data?select=train.csv)

In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
X['pj'] = X.weight * np.where((X.resp > 0), X.resp, 0)
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
X.drop('pj', axis=1, inplace=True)
u

224162.2681796676

## Split training set and fit tree

In [3]:
X = X[X.date > 85]
y = np.where((X.resp > 0), 1, 0)
X.drop(['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id'], axis=1, inplace=True)

In [4]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=100, verbose=1, random_state=0).fit(X, y)

Binning 1.757 GB of training data: 7.578 s
Binning 0.195 GB of validation data: 0.583 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.69280, val loss: 0.69285, in 0.549s
[2/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.69253, val loss: 0.69262, in 0.527s
[3/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.69228, val loss: 0.69239, in 0.509s
[4/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.69199, val loss: 0.69212, in 0.444s
[5/100] 1 tree, 31 leaves, max depth = 13, train loss: 0.69172, val loss: 0.69188, in 0.484s
[6/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.69148, val loss: 0.69166, in 0.546s
[7/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.69127, val loss: 0.69148, in 0.421s
[8/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.69110, val loss: 0.69133, in 0.511s
[9/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.69095, val loss: 0.69120, in 0.498s
[10/100] 1 tree, 31 leaves, max depth = 8, train 

In [5]:
del X, y

In [6]:
import pickle
with open('clf.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
X = pd.concat([example_sample_submission.set_index('ts_id'), example_test.set_index('ts_id')], axis=1)

In [ ]:
X

In [ ]:
X['pj'] = X.weight * X.action
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)

In [ ]:
X['pj'] = X.weight * np.where((X.resp > 0), X.resp, 0)
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)